# Seattle Housing Prices - Predictive Modeling

### Workflow:

1. Feature Engineering
    - (a) Create dummy variables
    - (b) Adjusting year_renovated column as categorical (yes or no) rather than by date
    - (c) Drop sqft_lot and sqft_lot15
    
    
2. Create Train_Test_Split


3. Run baseline model
    - print RMSE


4. Iterate on model:
    - (a) Remove outliers
    - (b) Remove collinear variables
    - (c) Log transformations
    - (d) Drop non-sig p-values
    - (e) Unit Vectorization (update x-columns to add in unit vectors composed of collinear variables)
    
 

### Plan so far...

#### Dependent Variable:

    - Price

#### Explanatory Variables:

    - Keep: ['bedrooms',
             'bathrooms',
             'sqft_living',
             'floors',
             'sqft_above',
             'sqft_basement',
             'sqft_living15',
             'view',
             'waterfront',
             'yr_built]
    
    - Drop: ['sqft_lot', 'sqft_lot15']
    
    - Create dummies: ['condition', 'zipcode', 'grade']
    
    - Convert to binary: ['yr_renovated']
    
    - Will have to think of a plan for latitude and longitude

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set_style('darkgrid')

from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

#### Import and drop columns:

In [3]:
house_data = pd.read_csv('house_data_cleaned.csv')
house_data.drop(['sqft_lot', 'sqft_lot15'], axis=1, inplace=True)
house_data.head()

,price,bedrooms,bathrooms,sqft_living,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15
0,221900.0,3,1.00,1180,1.0,0.0,0.0,3,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340
1,538000.0,3,2.25,2570,2.0,0.0,0.0,3,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690
2,180000.0,2,1.00,770,1.0,0.0,0.0,3,6,770,0.0,1933,0.0,98028,47.7379,-122.233,2720
3,604000.0,4,3.00,1960,1.0,0.0,0.0,5,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360
4,510000.0,3,2.00,1680,1.0,0.0,0.0,3,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800


In [4]:
# 17 columns going on 97...
house_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          21597 non-null  float64
 1   bedrooms       21597 non-null  int64  
 2   bathrooms      21597 non-null  float64
 3   sqft_living    21597 non-null  int64  
 4   floors         21597 non-null  float64
 5   waterfront     21597 non-null  float64
 6   view           21597 non-null  float64
 7   condition      21597 non-null  int64  
 8   grade          21597 non-null  int64  
 9   sqft_above     21597 non-null  int64  
 10  sqft_basement  21597 non-null  float64
 11  yr_built       21597 non-null  int64  
 12  yr_renovated   21597 non-null  float64
 13  zipcode        21597 non-null  int64  
 14  lat            21597 non-null  float64
 15  long           21597 non-null  float64
 16  sqft_living15  21597 non-null  int64  
dtypes: float64(9), int64(8)
memory usage: 2.8 MB


#### Create dummy variables:

In [5]:
# Create a binary renovated column to determine whether the home was renovated or not
# Since the year renovated is not actionable unless we specialize in selling renovated homes from specific years
# but this way we can determine whether we should focus on renovations or not
house_data['yr_renovated'] = house_data.yr_renovated.apply(lambda x: 1 if x > 0 else 0)
house_data.rename(columns = {"yr_renovated":"renovated"}, inplace=True)

In [6]:
# Drop first variable so they do not reference one another
cond_dum = pd.get_dummies(house_data['condition'], prefix='cond_', drop_first=True)
grade_dum = pd.get_dummies(house_data['grade'], prefix='grade_', drop_first=True)
zipcode_dum = pd.get_dummies(house_data['zipcode'], prefix='zipcode_', drop_first=True)
zipcode_dum.head()

,zipcode__98002,zipcode__98003,zipcode__98004,zipcode__98005,zipcode__98006,zipcode__98007,zipcode__98008,zipcode__98010,zipcode__98011,zipcode__98014,...,zipcode__98146,zipcode__98148,zipcode__98155,zipcode__98166,zipcode__98168,zipcode__98177,zipcode__98178,zipcode__98188,zipcode__98198,zipcode__98199
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Now to merge dummy columns onto our dataframe
upHouse_data = pd.concat([house_data, cond_dum, grade_dum, zipcode_dum], axis=1)
upHouse_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 100 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   price           21597 non-null  float64
 1   bedrooms        21597 non-null  int64  
 2   bathrooms       21597 non-null  float64
 3   sqft_living     21597 non-null  int64  
 4   floors          21597 non-null  float64
 5   waterfront      21597 non-null  float64
 6   view            21597 non-null  float64
 7   condition       21597 non-null  int64  
 8   grade           21597 non-null  int64  
 9   sqft_above      21597 non-null  int64  
 10  sqft_basement   21597 non-null  float64
 11  yr_built        21597 non-null  int64  
 12  renovated       21597 non-null  int64  
 13  zipcode         21597 non-null  int64  
 14  lat             21597 non-null  float64
 15  long            21597 non-null  float64
 16  sqft_living15   21597 non-null  int64  
 17  cond__2         21597 non-null

In [8]:
# Dropping un-dummied columns
upHouse_data.drop(columns=['condition', 'zipcode', 'grade'], inplace=True)
upHouse_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 97 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   price           21597 non-null  float64
 1   bedrooms        21597 non-null  int64  
 2   bathrooms       21597 non-null  float64
 3   sqft_living     21597 non-null  int64  
 4   floors          21597 non-null  float64
 5   waterfront      21597 non-null  float64
 6   view            21597 non-null  float64
 7   sqft_above      21597 non-null  int64  
 8   sqft_basement   21597 non-null  float64
 9   yr_built        21597 non-null  int64  
 10  renovated       21597 non-null  int64  
 11  lat             21597 non-null  float64
 12  long            21597 non-null  float64
 13  sqft_living15   21597 non-null  int64  
 14  cond__2         21597 non-null  uint8  
 15  cond__3         21597 non-null  uint8  
 16  cond__4         21597 non-null  uint8  
 17  cond__5         21597 non-null 

### Now Train-Test-Split

... in preparation for our baseline model

In [9]:
# Your code here - Define the target and predictors
outcome = 'price'
feat_cols = list(upHouse_data.columns)
feat_cols.remove(outcome)

In [10]:
X = upHouse_data[feat_cols]
y = upHouse_data['price']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 117)
print(len(X_train), len(y_test))
X_train.head()

15117 6480


,bedrooms,bathrooms,sqft_living,floors,waterfront,view,sqft_above,sqft_basement,yr_built,renovated,...,zipcode__98146,zipcode__98148,zipcode__98155,zipcode__98166,zipcode__98168,zipcode__98177,zipcode__98178,zipcode__98188,zipcode__98198,zipcode__98199
4900,3,1.00,1200,1.0,0.0,0.0,1200,0.0,1969,0,...,0,0,0,0,0,0,0,0,0,0
19664,3,2.50,1920,1.0,0.0,0.0,1920,0.0,2014,0,...,0,0,0,0,0,0,0,0,0,0
13949,3,1.75,1590,1.5,0.0,0.0,1590,0.0,1926,0,...,0,0,0,0,0,0,0,0,0,0
15272,4,4.00,2530,3.0,0.0,2.0,2100,430.0,2013,0,...,0,0,0,0,0,0,0,0,0,0
14411,3,2.50,2010,2.0,0.0,0.0,1390,620.0,2014,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
X_train = sm.add_constant(X_train)

In [57]:
X_test = sm.add_constant(X_test)

### Vanilla Model

In [13]:
predictors = '+'.join(feat_cols)
formula = outcome + '~' + predictors
model_vanilla = sm.OLS(y_train, X_train).fit()
model_vanilla.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.834
Model:                            OLS   Adj. R-squared:                  0.833
Method:                 Least Squares   F-statistic:                     783.8
Date:                Tue, 17 Nov 2020   Prob (F-statistic):               0.00
Time:                        15:34:03   Log-Likelihood:            -2.0141e+05
No. Observations:               15117   AIC:                         4.030e+05
Df Residuals:                   15020   BIC:                         4.038e+05
Df Model:                          96                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -3.308e+07   6.73e+06     -4.915      0.000   -4.63e+07   -1.99e+07
bedrooms       -1.139e+04   1696.973     -6.712      0.000   -1.47e+04   -8062.984
bathrooms        1.99e+04   2918.057      6.819      0.000    1.42e+04    2.56e+04
sqft_living       85.2973     15.316      5.569      0.000      55.277     115.318
floors         -2.728e+04   3535.648     -7.715      0.000   -3.42e+04   -2.03e+04
waterfront      6.679e+05   1.59e+04     42.105      0.000    6.37e+05    6.99e+05
view             5.53e+04   1909.117     28.966      0.000    5.16e+04     5.9e+04
sqft_above        80.3272     15.394      5.218      0.000      50.152     110.502
sqft_basement     32.4125     15.167      2.137      0.033       2.684      62.141
yr_built        -278.9553     72.601     -3.842      0.000    -421.263    -136.648
renovated       5.188e+04   7097.815      7.309      0.000     3.8e+04    6.58e+04
lat              2.13e+05   7.03e+04      3.030      0.002    7.52e+04    3.51e+05
long           -1.928e+05   4.95e+04     -3.897      0.000    -2.9e+05   -9.58e+04
sqft_living15     14.2867      3.225      4.430      0.000       7.965      20.608
cond__2         7.923e+04   3.62e+04      2.188      0.029    8265.549     1.5e+05
cond__3         6.844e+04   3.36e+04      2.035      0.042    2528.473    1.34e+05
cond__4         9.203e+04   3.36e+04      2.736      0.006    2.61e+04    1.58e+05
cond__5         1.414e+05   3.38e+04      4.179      0.000    7.51e+04    2.08e+05
grade__4       -1.097e+05   1.53e+05     -0.718      0.473   -4.09e+05     1.9e+05
grade__5       -1.297e+05   1.49e+05     -0.868      0.385   -4.23e+05    1.63e+05
grade__6       -1.332e+05   1.49e+05     -0.894      0.371   -4.25e+05    1.59e+05
grade__7       -1.286e+05   1.49e+05     -0.863      0.388   -4.21e+05    1.63e+05
grade__8       -1.067e+05   1.49e+05     -0.716      0.474   -3.99e+05    1.85e+05
grade__9       -3.002e+04   1.49e+05     -0.201      0.841   -3.22e+05    2.62e+05
grade__10       8.848e+04   1.49e+05      0.593      0.553   -2.04e+05    3.81e+05
grade__11       3.109e+05    1.5e+05      2.078      0.038    1.76e+04    6.04e+05
grade__12       7.684e+05   1.51e+05      5.093      0.000    4.73e+05    1.06e+06
grade__13       1.165e+06   1.59e+05      7.337      0.000    8.54e+05    1.48e+06
zipcode__98002  1.802e+04   1.59e+04      1.136      0.256   -1.31e+04    4.91e+04
zipcode__98003 -9189.1622   1.42e+04     -0.645      0.519   -3.71e+04    1.87e+04
zipcode__98004  7.374e+05    2.6e+04     28.318      0.000    6.86e+05    7.88e+05
zipcode__98005  2.672e+05   2.77e+04      9.644      0.000    2.13e+05    3.22e+05
zipcode__98006  2.208e+05   2.26e+04      9.764      0.000    1.77e+05    2.65e+05
zipcode__98007  2.131e+05   2.84e+04      7.503      0.000    1.57e+05    2.69e+05
zipcode__980

In [ ]:
# We see a very high R-squared and large standard errors, likely because of collinearity
# During our EDA, we noted there were several collinear variable pairs to be addressed

### Side Note: Model Function

In [70]:
def training_model(features, outcome, y_train, X_train):
    predictors = '+'.join(features)
    formula = outcome + '~' + predictors
    model_vanilla = sm.OLS(y_train, X_train).fit()
    return model_vanilla.summary()

With our QQ plot for the vanilla model above, we can see that the graph is not normal with heavy skew on both low and high ends. The upper end has a greater skew showing there are likely a large number of outliers on this end.

We may want to consider reducing outliers at 2 standard deviations (95%) to have the most accurate predictions. The only reason we wouldn't is if we wanted to focus on selling either cheap or luxury homes. If we were doing so, we would have to narrow our data set to these alone.

In [52]:
y_hat_train = model_vanilla.predict(X_train)

4900     1.592513e+05
19664    3.129222e+05
13949    2.914685e+05
15272    1.032851e+06
14411    4.732969e+05
             ...     
15969    1.137740e+05
20055    5.313008e+05
6224     2.722516e+05
9328     2.204981e+05
11758    1.342246e+06
Length: 15117, dtype: float64

In [64]:
y_hat_train = model_vanilla.predict(X_train)
rmse_train = mse(y_train, y_hat_train)**0.5
y_hat_test = model_vanilla.predict(X_test)
rmse_test = mse(y_test, y_hat_test)**0.5
print(f'Train RMSE: {rmse_train} \n Test RMSE: {rmse_test}')
print('Percent change: ', round(((rmse_test-rmse_train)/rmse_train)*100, 3))


Train RMSE: 147910.53759775453 
 Test RMSE: 157970.48914323247
Percent change:  6.801


In [ ]:
# Our test model is actually not far off, especially for the vanilla model.
# That being said, RMSE seems high. I think we can bring this down...
# especially by removing outliers and collinearity

### Model 2: Removing Outliers

In [69]:
training_model(feat_cols, outcome, y_train, X_train)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.834
Model:                            OLS   Adj. R-squared:                  0.833
Method:                 Least Squares   F-statistic:                     783.8
Date:                Tue, 17 Nov 2020   Prob (F-statistic):               0.00
Time:                        16:53:16   Log-Likelihood:            -2.0141e+05
No. Observations:               15117   AIC:                         4.030e+05
Df Residuals:                   15020   BIC:                         4.038e+05
Df Model:                          96                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -3.308e+07   6.73e+06     -4.915      0.000   -4.63e+07   -1.99e+07
bedrooms       -1.139e+04   1696.973     -6.712      0.000   -1.47e+04   -8062.984
bathrooms        1.99e+04   2918.057      6.819      0.000    1.42e+04    2.56e+04
sqft_living       85.2973     15.316      5.569      0.000      55.277     115.318
floors         -2.728e+04   3535.648     -7.715      0.000   -3.42e+04   -2.03e+04
waterfront      6.679e+05   1.59e+04     42.105      0.000    6.37e+05    6.99e+05
view             5.53e+04   1909.117     28.966      0.000    5.16e+04     5.9e+04
sqft_above        80.3272     15.394      5.218      0.000      50.152     110.502
sqft_basement     32.4125     15.167      2.137      0.033       2.684      62.141
yr_built        -278.9553     72.601     -3.842      0.000    -421.263    -136.648
renovated       5.188e+04   7097.815      7.309      0.000     3.8e+04    6.58e+04
lat              2.13e+05   7.03e+04      3.030      0.002    7.52e+04    3.51e+05
long           -1.928e+05   4.95e+04     -3.897      0.000    -2.9e+05   -9.58e+04
sqft_living15     14.2867      3.225      4.430      0.000       7.965      20.608
cond__2         7.923e+04   3.62e+04      2.188      0.029    8265.549     1.5e+05
cond__3         6.844e+04   3.36e+04      2.035      0.042    2528.473    1.34e+05
cond__4         9.203e+04   3.36e+04      2.736      0.006    2.61e+04    1.58e+05
cond__5         1.414e+05   3.38e+04      4.179      0.000    7.51e+04    2.08e+05
grade__4       -1.097e+05   1.53e+05     -0.718      0.473   -4.09e+05     1.9e+05
grade__5       -1.297e+05   1.49e+05     -0.868      0.385   -4.23e+05    1.63e+05
grade__6       -1.332e+05   1.49e+05     -0.894      0.371   -4.25e+05    1.59e+05
grade__7       -1.286e+05   1.49e+05     -0.863      0.388   -4.21e+05    1.63e+05
grade__8       -1.067e+05   1.49e+05     -0.716      0.474   -3.99e+05    1.85e+05
grade__9       -3.002e+04   1.49e+05     -0.201      0.841   -3.22e+05    2.62e+05
grade__10       8.848e+04   1.49e+05      0.593      0.553   -2.04e+05    3.81e+05
grade__11       3.109e+05    1.5e+05      2.078      0.038    1.76e+04    6.04e+05
grade__12       7.684e+05   1.51e+05      5.093      0.000    4.73e+05    1.06e+06
grade__13       1.165e+06   1.59e+05      7.337      0.000    8.54e+05    1.48e+06
zipcode__98002  1.802e+04   1.59e+04      1.136      0.256   -1.31e+04    4.91e+04
zipcode__98003 -9189.1622   1.42e+04     -0.645      0.519   -3.71e+04    1.87e+04
zipcode__98004  7.374e+05    2.6e+04     28.318      0.000    6.86e+05    7.88e+05
zipcode__98005  2.672e+05   2.77e+04      9.644      0.000    2.13e+05    3.22e+05
zipcode__98006  2.208e+05   2.26e+04      9.764      0.000    1.77e+05    2.65e+05
zipcode__98007  2.131e+05   2.84e+04      7.503      0.000    1.57e+05    2.69e+05
zipcode__980